This is a "lite" version of the ordering script we are now using. 

We found it easier for the team to use, rather than command line (I was using it myself at first). 

New features include ordering samples only even when stock is available so we don't have to break boxes.

Name of suppliers have been removed to "SupplierN"

In [ ]:
print(" Panick only if nothing happens after 30 seconds :) ")

# Import libraries
import pandas as pd
import math
import datetime
from pathlib import Path


# Open the supplier's price list
suppliers_pricelist = pd.read_csv('All Suppliers Price List - update 300622.csv')

# Open and read the daily order file
dir_to_downloads = '/Users/Delphine/downloads'   # Change the path
p_downloads = Path(dir_to_downloads)
order = pd.read_csv(p_downloads/'Orders_17566_2022-08-11_150824.csv') # Daily change of the file's name 



### CATCHING SAMPLES
for i in range (0, len(order)):
    try:
        sku = order['Line item sku'][i]
        sku_index = suppliers_pricelist[suppliers_pricelist['Item No.'] == sku].index[0]
        suppliers_pricelist.loc[sku_index, 'MOQ']
        order.loc[i, 'supplier MOQ'] =  suppliers_pricelist.loc[sku_index, 'MOQ']
        order.loc[i, 'Unit'] =  suppliers_pricelist.loc[sku_index, 'Unit']
    
    # In case the product isn't on the price list
    except IndexError: 
        pass

# Create a dataframe for the samples only and grab the indexes to remove them from the order df
col = ['Variant Sku', 'Variant Inventory Qty']
samples = pd.DataFrame(columns=col)
index_to_drop = []

for i in range (0, len(order)):
    if order['Line item quantity'][i] == 1 and order['supplier MOQ'][i] >1 : 
        samples = samples.append({'Variant Sku' : order['Line item sku'][i],
                                 'Variant Inventory Qty' : order['Product variant inventory quantity'][i] },  
                                 ignore_index=True )
        index_to_drop.append(i) 
        
    if order['Line item quantity'][i] == 1 and order['Unit'][i] == 'Doz':
        samples = samples.append({'Variant Sku' : order['Line item sku'][i],
                                 'Variant Inventory Qty' : order['Product variant inventory quantity'][i]},  
                                 ignore_index=True )
        index_to_drop.append(i) 


# remove samples from the orders so they are not processed like a normal order
order.drop(order.index[index_to_drop], inplace=True )


# Samples inventory update

samples['count'] = 1
samples_count = samples.groupby('Variant Sku', as_index=False).sum()
samples_raw = samples.drop_duplicates(subset='Variant Sku').drop('count', axis=1).sort_values(by=['Variant Sku'])
samples_inv = pd.merge(samples_count, samples_raw, on='Variant Sku')

for i in range(0, len(samples_inv)):
    samples_inv.loc[i, 'Variant Inventory Qty'] = samples_inv['Variant Inventory Qty'][i] + samples_inv['count'][i]
    
samples_inv = samples_inv.drop('count', axis = 1)

### ENDS SAMPLES CODE


# sub df that will take all products needed and add up the duplicates
sku_needed = order[['Line item sku', 'Line item quantity']]
sku_needed = sku_needed.groupby('Line item sku', as_index=False).sum()

# sub df that will take the skus and inventory and remove the duplicates
sku_inventory = order[['Line item sku', 'Product variant inventory quantity']]
sku_inventory = sku_inventory.drop_duplicates(subset='Line item sku')

# new df, merging both the sku_needed and sku_inventory df
ready_to_process = pd.merge(sku_needed, sku_inventory, on='Line item sku')

# Create the to_order df
columns = ['Variant Sku', 'Qty_needed', 'MOQ_order', 'Variant Inventory Qty', 'Supplier']
to_order = pd.DataFrame(columns=columns)

# Get today's date to create the PO number
x = datetime.datetime.now()
d = x.strftime('%d')
m = x.strftime('%m')
y = x.strftime('%y')


# add the skus and quantities needed to the df
for i in range(0, len(ready_to_process)):
    if ready_to_process['Product variant inventory quantity'][i] < 0 :
        
        if abs(ready_to_process['Product variant inventory quantity'][i]) == ready_to_process['Line item quantity'][i]:
            to_order = to_order.append({'Variant Sku': ready_to_process['Line item sku'][i], 
                                        'Qty_needed': ready_to_process['Line item quantity'][i], 
                                        'Shopify_inventory': ready_to_process['Product variant inventory quantity'][i],
                                        'Supplier' : 'tbd'}, 
                                        ignore_index=True)
                                
        elif abs(ready_to_process['Product variant inventory quantity'][i]) < ready_to_process['Line item quantity'][i]: 
            to_order = to_order.append({'Variant Sku':ready_to_process['Line item sku'][i], 
                                        'Qty_needed': abs(ready_to_process['Product variant inventory quantity'][i]), 
                                        'Shopify_inventory': ready_to_process['Product variant inventory quantity'][i],
                                        'Supplier' : 'tbd'}, 
                                        ignore_index=True)
                                           
        elif abs(ready_to_process['Product variant inventory quantity'][i]) > ready_to_process['Line item quantity'][i]:
            to_order = to_order.append({'Variant Sku':ready_to_process['Line item sku'][i], 
                                        'Qty_needed': abs(ready_to_process['Line item quantity'][i]), 
                                        'Shopify_inventory': ready_to_process['Product variant inventory quantity'][i],
                                        'Supplier' : 'tbd'}, 
                                       ignore_index=True)
            
    else: # if the inventory is at 0, we don't need to process.
        pass 


# Include the increments for accurate order placed with the suppliers and modify the new inventory accordingly

for i in range (0, len(to_order)):
    sku = to_order['Variant Sku'][i]
    qty_needed = to_order['Qty_needed'][i]
    old_inv = to_order['Shopify_inventory'][i]
    
    

    try:
        # find the sku index to order in the supplier's price list
        sku_index = suppliers_pricelist[suppliers_pricelist['Item No.'] == sku].index[0]
        # find the sku's unit in the supplier's price list
        sku_unit = suppliers_pricelist['Unit'][sku_index]
        # find the MOQ in the suppliers's price list
        sku_moq = suppliers_pricelist['MOQ'][sku_index]
        # add matching supplier
        sku_supplier = suppliers_pricelist['Supplier'][sku_index]     
    


        # Cutlery and products sold by Dozens
        if sku_unit == 'Doz':
            quantity_to_order = math.ceil(qty_needed/12)
            new_inv = old_inv + quantity_to_order*12
           
             

       # For the rest of the products     
        else :
            quantity_to_order = math.ceil(qty_needed/sku_moq)*sku_moq         
            new_inv = old_inv + quantity_to_order
            

    # When the product code isn't availble on the price list        
    except IndexError: 
        quantity_to_order = to_order['Qty_needed'][i]
        new_inv = old_inv + quantity_to_order
        sku_supplier = 'TBD'
        
    
    # Add the quantities to order, new inventory and matching suppliers for each sku   
    to_order.at[i, 'MOQ_order'] = quantity_to_order
    to_order.at[i, 'Variant Inventory Qty'] = new_inv
    to_order.at[i, 'Supplier'] = sku_supplier


# Save a csv file to import to excelify

inventory_to_import = to_order[['Variant Sku', 'Variant Inventory Qty']]
new_inv = inventory_to_import.append(samples_inv)
new_inv.to_csv('new-inventory.csv', index=False)
print('\n new-inventory.csv file saved and new inventory ready for import on Excelify')

# Prepare dataframes for each suppliers

Supplier1 = to_order.loc[to_order['Supplier'] == 'Supplier1'].reset_index(drop=True)
Supplier2 = to_order.loc[to_order['Supplier'] == 'Supplier2'].reset_index(drop=True)
Supplier3 = to_order.loc[to_order['Supplier'] == 'Supplier3'].reset_index(drop=True)
Supplier4 = to_order.loc[to_order['Supplier'] == 'Supplier4'].reset_index(drop=True)
Sort = to_order.loc[to_order['Supplier'] == 'TBD'].reset_index(drop=True)

# Print orders ready for copy and paste
if len(samples) > 0:
    print('\n *** Samples on order today :\n' )
    for i in range(0, len(samples)):
        print(samples['Variant Sku'][i])
else:
    print('no samples today')

# OTHER's, products that don't match the price list 
if len(Sort) > 0:
    print('\n  ### OTHER PRODUCTS - TO ALLOCATE TO RIGHT SUPPLIER #### \n')
    for i in range(0, len(Sort)):
        print(Sort['Variant Sku'][i], 'x', int(Sort['MOQ_order'][i]))
else:
    pass

# Supplier3's PO    
if len(Supplier3) > 0:    
    print('\n   ### Supplier3 PO #### \n')
    print('PO-TR{}{}{} \n'.format(d,m,y))
    print('Hi Supplier3! \n\nCan we please order the following for pickup at 10 tomorrow:\n')
    for i in range(0, len(Supplier3)):
        print(Supplier3['Variant Sku'][i], 'x', int(Supplier3['MOQ_order'][i]))
    print('\nThank you!\n')
else:
    pass

# Supplier2's PO
if len(Supplier2) > 0:
    print('\n   ### Supplier2 PO #### \n')
    print('PO-TK{}{}{} \n'.format(d,m,y))
    print('Hi Supplier2! \n\nCan you please deliver the following to our warehouse tomorrow:\n')
    for i in range(0, len(Supplier2)):
        print(Supplier2['Variant Sku'][i].replace('.TK',' '), 'x', int(Supplier2['MOQ_order'][i]))

    print('\nThank you!')
else:
    pass

if len(Supplier4) > 0:
# Supplier4's PO
    print('\n   ### Supplier4 PO #### \n')
    print('PO-BP{}{}{} \n'.format(d,m,y))
    print('Hi Supplier4! \n\nCan you please deliver the following to our warehouse :\n')
    for i in range(0, len(Supplier4)):
        print(Supplier4['Variant Sku'][i], 'x', int(Supplier4['MOQ_order'][i]))
    print('\nThank you!')
else:
    pass



# Supplier1's PO    
if len(Supplier1) > 0:    
    print('\n   ### TSupplier1 PO #### \n')
    print('PO TRWA{}{}{} \n'.format(d,m,y))
    print('Hi Team! \n\n Could you please add the following to the next palet going to Supplier2 in Sydney, we will pick it up from there:\n')
    for i in range(0, len(Supplier1)):
        print(Supplier1['Variant Sku'][i], 'x', int(Supplier1['MOQ_order'][i]))
    print('\nThank you!\n')
else:
    pass